In [364]:
#INITIALIZA THE FUNCTIONS 
#%%
#import the necessary packages
import automatization
from automatization.ple_statistics import PleAuto
import pandas as pd
import os
import time
import json
from importlib import reload
import numpy as np
from automatization.ple_statistics import PleAuto
from matplotlib import pyplot as plt
reload(automatization.ple_statistics)
from automatization.ple_statistics import PleAuto
from scipy.constants import speed_of_light
import pickle
from pathlib import Path

import sys

sys.setrecursionlimit(10000)

pa = PleAuto(
    ple_gui, 
    ple_optimize_logic, 
    laser_scanner_logic,
    poi_manager_logic, 
    scanning_optimize_logic,
    spectrometer,  
    
    spectrometerlogic,
    cobolt,
    ibeam_smart
)

# def ple_is_here(res, center_sigma = 3e3, amplitude = 1000, sigma_stderr_ratio = 4, amplitude_stderr_ratio=3):
#     """Check if the ple is still there."""
#     it_is = True
#     if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
#         return False
#     if ( res["center"].stderr > center_sigma or 
#     res["sigma"].stderr * sigma_stderr_ratio > res["sigma"].value or 
#     res["amplitude"].stderr * amplitude_stderr_ratio > res["amplitude"].value or
#     res["amplitude"].value < amplitude): 
#         #ple is gone.
#         return False

#     return it_is
def ple_is_here(rsquared_bound = 0.5):
    """Check if the ple is still there."""
    
    if ple_gui.fit_result[1].rsquared < rsquared_bound:
        print("NO, PLE", ple_gui.fit_result[1].rsquared )
        return False
    
    else:
        return True


def adjust_eta(pa, poi_name, folder_defect, results_poi, center_v):
    """Adjust the etalon voltage to get the best fit."""
    # ple_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["a"])
    ple_gui._mw.frequencyDoubleSpinBox.setValue(1000)
    
    eta_volts = [center_v, center_v + 0.2, center_v + 0.4, center_v + 0.6, center_v - 0.2, center_v - 0.4,  center_v - 0.6]
    r_squared = []
    for eta_v in eta_volts:
        laser_controller_remote.etalon_voltage = eta_v
        time.sleep(0.5)
        res = pa.do_ple_scan(lines = 1)
        time.sleep(0.5)
        r_squared.append(ple_gui.fit_result[1].rsquared)
    results_poi["eta_voltage"] = eta_volts[r_squared.index(max(r_squared))]
    laser_controller_remote.etalon_voltage = eta_volts[r_squared.index(max(r_squared))]
    res = pa.do_ple_scan(lines = 1)
    pa.save_ple(tag = "full_range_eta_adjusted",
                poi_name=poi_name, folder_name=folder_defect)

    ple_gui._mw.frequencyDoubleSpinBox.setValue(250)
    return res, results_poi
def ple_refocus(pa, opt_times = 1, 
                scan_frequency=200, 
                scan_resolution=500,
                scan_range = 5000):
    
    seqs = {str(seq): idx for idx, seq in enumerate(ple_gui._osd.settings_widget.available_opt_sequences)}
    ple_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["a"])
    ple_gui._osd.change_settings({'scan_frequency': {"a": scan_frequency},
                                    "scan_resolution": {"a":scan_resolution},
                                    "scan_range": {"a": scan_range}}) #GHz

    ple_gui._osd.accept()
    time.sleep(0.8)
    pa.optimize_ple()
    return res
def settings_confocal_refocus_fine():
    seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["xy, z"])
    scanner_gui._osd.change_settings({'scan_frequency': {"x": 25, "y": 25, "z": 25},
                                    "scan_resolution": {"x": 25, "y": 25, "z":80},
                                    "scan_range": {"x": 1e-6, "y": 1e-6, "z": 4e-6}})

    scanner_gui._osd.accept()
    time.sleep(0.5)


def settings_confocal_refocus_coarse():
    seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["x, y, z"])

    scanner_gui._osd.change_settings({'scan_frequency': {"x": 5, "y": 5, "z": 5},
                                    "scan_resolution": {"x": 80, "y": 80, "z":80},
                                    "scan_range": {"x": 2.5e-6, "y": 2.5e-6, "z": 4.5e-6}})
    scanner_gui._osd.accept()
    time.sleep(0.5)

def confocal_refocus(opt_times=2):
    for i in range(opt_times):
        scanning_optimize_logic.start_optimize()
        while scanning_optimize_logic.module_state()=='locked':
            time.sleep(0.5)
    time.sleep(0.5)
#find the defect:
def find_the_defect(pa, poi_name, folder_defect):
    ple_gui._mw.frequencyDoubleSpinBox.setValue(1000)
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    pa.set_resonant_power(power = 300)
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(2)
    time.sleep(1)
    settings_confocal_refocus_coarse()
    confocal_refocus(opt_times=2)

    
    switchlogic.set_state("ScanningMode", 'NI')
    time.sleep(0.5)
    #Check how the PLE look like
    res = pa.do_ple_scan(lines = 1)

    #configure slow scanning for the wavemeter scanning optimizations
    for kk in range(3):
        if not ple_is_here():
            switchlogic.set_state("ScanningMode", 'Wavemeter')
            time.sleep(0.5)
            settings_confocal_refocus_coarse()
            confocal_refocus(opt_times=1)
            #Check how the PLE look like
            time.sleep(0.5)
            switchlogic.set_state("ScanningMode", 'NI')
            res = pa.do_ple_scan(lines = 1)
            time.sleep(0.5)
            pa.save_ple(tag = "full_range_iter_{kk}",
                    poi_name=poi_name, folder_name=folder_defect)
            time.sleep(0.5)
        else:
            break

    pa.save_ple(tag = "full_range",
            poi_name=poi_name, 
            folder_name=folder_defect)
    time.sleep(0.2)
    ple_gui._mw.frequencyDoubleSpinBox.setValue(250)
    return res

def take_spectrum(pa, poi_name, folder_defect, results_poi):
    # take spectrum to estimate SOC
    pa.set_resonant_power(power = 0)
    time.sleep(2)
    ibeam_smart.enable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 100)
    pa.set_resonant_power(power = 0)
    time.sleep(1)
    pa.take_spectrum()

    
    
    time.sleep(1)
    spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
    time.sleep(1)
    pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
    # spectrometer.fit_results.params["center_1"].value
    if spectrometer.fit_results is not None:
        params = spectrometer.fit_results.params
        results_poi["SOC, GHz"] = float(speed_of_light / params["center_2"].value - speed_of_light / params["center_1"].value)
        results_poi["center_1, GHz"] = float(params["center_1"].value)
        results_poi["center_2, GHz"] = float(params["center_2"].value)
    ibeam_smart.disable()
    #cobolt.disable_modulated()
    time.sleep(0.5)
    results_poi["spectrum_data"] = spectrometerlogic.last_saved_path
    # results_poi["spectrum_fit_data"] = spectrometer.fit_results.best_fit
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 2)
    pa.set_resonant_power(power = 300)    
    time.sleep(1)
    return results_poi
# Perform the saturation measurement

def fine_optimize(pa, poi_name, folder_defect, results_poi):
    res = pa.do_ple_scan(lines = 1)
    pa.go_to_ple_target(res["center"].value)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    settings_confocal_refocus_fine()
    confocal_refocus(opt_times=1)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    confocal_refocus(opt_times=1)
    res = pa.do_ple_scan(lines = 1)
    pa.go_to_ple_target(res["center"].value)
    results_poi["center"] = res["center"].value
    
    results_poi["center_λ"] = high_finesse_wavemeter_remote.get_current_wavelength()
    pa.save_ple(tag = "full_range_optimized",
            poi_name=poi_name, 
            folder_name=folder_defect)
    return res, results_poi

def run_saturation_measurement(pa, res, poi_name, folder_defect, results_poi, large_sigma = 4000, small_sigma = 300, power_steps = None, low_power_steps = None, repumped_power = 100, no_repump_power = 150):
    ple_gui._mw.frequencyDoubleSpinBox.setValue(500)

    os.makedirs(saturation_folder := os.path.join(folder_defect, "saturation"), exist_ok=True)
    results_poi["saturation"] = {}
    idx_no_ple = None
    res_old = res
    if power_steps is None:
        power_steps = 3 * np.logspace(1.5, 2, 10, endpoint=True).astype(int)[::-1]
    if low_power_steps is None:
        low_power_steps = np.array([85, 78, 70, 65, 60])
    power_steps = np.append(power_steps,low_power_steps)
    zero_range = (
                res["center"].value - large_sigma,
                res["center"].value + large_sigma
            )
    res = pa.do_ple_scan(lines = 1, in_range=zero_range)
    for idx, power in enumerate(power_steps):
        os.makedirs(power_folder := os.path.join(saturation_folder, f"{power}"), exist_ok=True)
        
        if power > repumped_power: 
            cobolt.enable_modulated()
            pa.set_resonant_power(power = power)
            time.sleep(1)
            #align twice
            # for i in range(2)
            
            fine_range = (
                res["center"].value - small_sigma,
                res["center"].value + small_sigma
            )
            if res["center"].value is None or res["sigma"].value is None or not ple_is_here(rsquared_bound=0.2):
                res = pa.do_ple_scan(lines = 1, in_range=zero_range)
                
                if not ple_is_here(rsquared_bound=0.2):
                    continue
            res = pa.do_ple_scan(lines = 1, in_range=fine_range)
            # if abs(res_old["center"].value - res["center"].value) > res_old["sigma"].value*2:
            #     res = res_old
            # else:s
            #     res_old = res
            pa.save_ple(tag = f"{power}",
                poi_name=poi_name, folder_name=power_folder)

            results_poi.update({"saturation": 
                            {f"{power}_repump":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr,
                                "center": res["center"].value
                                }
                                }})
        if (power < no_repump_power):
            os.makedirs(power_folder_norepump := os.path.join(power_folder, f"no_repump"), exist_ok=True)
            # check with initioalization
            cobolt.disable_modulated()
            pa.one_pulse_repump("violet")
            for i in range(3):
                pa.one_pulse_repump("violet")
                res_ = pa.do_ple_scan(lines = 1, in_range=fine_range)
                if not ple_is_here(rsquared_bound=0.4):
                    continue
            if not ple_is_here(rsquared_bound=0.4):
                    continue
            res_ = pa.do_ple_scan(lines = 5, in_range=fine_range)
            print("Saving no repump")
            pa.save_ple(tag = f"{power}_norepump",
                poi_name=poi_name, folder_name=power_folder_norepump)
            results_poi.update({"saturation": 
                            {f"{power}_norepump":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res_["sigma"].value,
                                "sigma_stderr": res_["sigma"].stderr,
                                "center": res_["center"].value
                                }
                                }})
        if power <= 40:
            breaks

            #save_plots
    return results_poi
# constraints
scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range
scanning_optimize_logic._backwards_line_resolution = 20


In [365]:
from matplotlib import pyplot as plt
from matplotlib import ticker
SMALL_SIZE = 15
MEDIUM_SIZE = 20
BIGGER_SIZE = 30

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
plt.rc('grid', linestyle='--')
plt.rc('grid', color='#E68F6B')
from ast import literal_eval as make_tuple
def get_scan_range(file_name):
    range_ = ""
    with open(file_name, "r") as f:
        for line in f.readlines():
            if "scan range" in line:
                range_ = line
    return make_tuple(range_.strip().split("=")[-1])


In [4]:
from datetime import datetime
print(datetime.today().strftime('%d-%m-%Y'))

30-04-2023


In [5]:
laser_controller_remote.etalon_voltage

-7.4

In [ ]:
folder = rf"C:\Users\yy3\Documents\data\Vlad\158\30-04\#1D\ROI1\{datetime.today().strftime('%d-%m-%Y')}"
os.makedirs(folder, exist_ok=True)
folder = os.path.join(folder, r"auto")

center_v = laser_controller_remote.etalon_voltage
os.makedirs(folder, exist_ok=True)
high_finesse_wavemeter_remote.start_acquisition()
# run throught the defects:
cobolt.enable_modulated()
for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    results_poi[poi_name] = {}  
    
    os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)
    #Go to the defect:
    cobolt.set_laser_modulated_power(10)
    settings_confocal_refocus_fine()
    pa.go_to_poi(poi_name, opt_times=0, ref_poi=None) 
    #
    print("Find the defect ", poi_name)
    res = find_the_defect(pa, poi_name, folder_defect)

    print("Run spectrum")
    results_poi = take_spectrum(pa, poi_name, folder_defect, results_poi)

    print("Run the saturation measurements")
    cobolt.set_laser_modulated_power(5)
    
    if not ple_is_here():
        print("Adjusting the eta")
        res, results_poi = adjust_eta(pa, poi_name, folder_defect, results_poi, center_v)
    if not ple_is_here():
        # return the center eta
        laser_controller_remote.etalon_voltage = center_v
        continue
    print("Fine optimization of the ple and confocal")
    res, results_poi = fine_optimize(pa, poi_name, folder_defect, results_poi)
    print("Run the saturation measurements")
    results_poi = run_saturation_measurement(pa, res, poi_name, folder_defect, results_poi)
    
    with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
        pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)

Find the defect  def3
Run spectrum
Run the saturation measurements
Fine optimization of the ple and confocal
Run the saturation measurements
Find the defect  def4
Run spectrum
Run the saturation measurements
Adjusting the eta
Fine optimization of the ple and confocal
Run the saturation measurements
Find the defect  def5
Run spectrum
Run the saturation measurements
Fine optimization of the ple and confocal
Run the saturation measurements
Find the defect  def6
Run spectrum
Run the saturation measurements
Fine optimization of the ple and confocal
Run the saturation measurements
Find the defect  def7
Run spectrum
Run the saturation measurements
Fine optimization of the ple and confocal
Run the saturation measurements
Find the defect  def8
Run spectrum
Run the saturation measurements
Fine optimization of the ple and confocal
Run the saturation measurements
Find the defect  def9
Run spectrum
Run the saturation measurements
Adjusting the eta
Find the defect  def10
Run spectrum
Run the saturat

KeyboardInterrupt: 

In [ ]:
spec_dat = np.genfromtxt(results_poi[poi_name]["spectrum_data"])
ax = plt.subplot(111)

ax.plot(spec_dat[:,0]/1e9, spec_dat[:,1])
ax.grid(linestyle='--', color='#E68F6B')
ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
ax.set_xlabel(r'$\lambda$, nm')
ax.set_ylabel(r'integrated counts')

In [10]:
# constraints
scan_range_constr = laser_scanner_logic.scan_ranges["a"] #laser_scanner_logic.scanner_constraints.axes["a"].value_range

In [11]:
scan_range_constr

(17853.0, 18202.0)

In [1]:
ibeam_smart.enable()

In [1]:
high_finesse_wavemeter_remote.start_acquisition()

0

In [12]:
scan_range_constr

(17853.0, 18202.0)

In [13]:
# factor_sigma = 0.509
pa.go_to_ple_target(scan_range_constr[0])
# time.sleep(5)


In [14]:
w1 = high_finesse_wavemeter_remote.get_current_wavelength()

In [15]:

pa.go_to_ple_target(scan_range_constr[1])



In [16]:
w2 = high_finesse_wavemeter_remote.get_current_wavelength()

factor_sigma = (w2 - w1) * 1000000 / (scan_range_constr[1] - scan_range_constr[0]) #GHz 

In [17]:
factor_sigma

0.9743837832557048

In [ ]:
factor_sigma = (w2 - w1) * 1000000 / laser_scanner_logic.scanner_constraints.axes["a"].value_range[1] #GHz

In [1]:
factor_sigma

NameError: name 'factor_sigma' is not defined

In [ ]:
laser_scanner_logic.scanner_constraints.axes["a"].value_range[1]

11473.132296146105

In [ ]:
ni_laser_scanner._update_position_ranges({"a": (0, laser_scanner_logic.scanner_constraints.axes["a"].value_range[1] * factor_sigma)})

In [ ]:
ni_laser_scanner._update_position_ranges({"a": (0, 26000 * factor_sigma)})

In [ ]:
settings_confocal_refocus_fine()

In [6]:
ibeam_smart.enable()

In [7]:
ibeam_smart.disable()

In [3]:
cobolt.set_laser_modulated_power(70)

In [8]:
cobolt.disable_modulated()

In [4]:
cobolt.enable_modulated()

In [2]:
settings_confocal_refocus_coarse()

KeyboardInterrupt: 

: 

: 

: 

In [ ]:
def calibrate_ple_scanner_wm():
    new_position_ranges = ()
    ni_laser_scanner._update_position_ranges()

In [7]:
from scipy.constants import c

In [12]:
w1 = high_finesse_wavemeter_remote.get_current_wavelength()
print(1e-3 * c / w1, "nm")

619.236112267036 nm


In [15]:
pa.one_pulse_repump("violet")

In [21]:
results_poi = {}

In [23]:
poi_name = "def5"

results_poi[poi_name] = {}  
folder = rf"C:\Users\yy3\Documents\data\Vlad\158\30-04\#1D"
os.makedirs(folder, exist_ok=True)

os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)
folder_defect = os.path.join(folder_defect, r"auto")

results_poi = take_spectrum(pa, poi_name, folder_defect, results_poi)

In [41]:
res = pa.do_ple_scan(lines = 1)

In [48]:
results_poi = run_saturation_measurement(pa, res, poi_name, folder_defect, results_poi, 
                                         power_steps=[300, 250, 220, 180, 150, 130, 120, 110, 100],
                                         repumped_power=120,
                                         no_repump_power=140)

Saving no repump
Saving no repump
Saving no repump
Saving no repump
Saving no repump
NO, PLE 0.007139380938749995
NO, PLE 0.007139380938749995
NO, PLE 0.013274512569453556
Saving no repump
Saving no repump
Saving no repump


In [50]:
cobolt.enable_modulated()

In [27]:
ple_gui.fit_result[1].rsquared

0.8446843850145778

In [ ]:
 
with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
    pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [49]:
folder = rf"C:\Users\yy3\Documents\data\Vlad\158\30-04\#1D\Auto"
os.makedirs(folder, exist_ok=True)
#folder = os.path.join(folder, r"auto2")

center_v = laser_controller_remote.etalon_voltage
os.makedirs(folder, exist_ok=True)
high_finesse_wavemeter_remote.start_acquisition()
# run throught the defects:
cobolt.enable_modulated()
for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    results_poi[poi_name] = {}  
    
    os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)
    #Go to the defect:
    cobolt.set_laser_modulated_power(5)
    settings_confocal_refocus_fine()
    pa.go_to_poi(poi_name, opt_times=0, ref_poi=None) 
    #
    print("Find the defect ", poi_name)
    res = find_the_defect(pa, poi_name, folder_defect)

    print("Run spectrum")
    results_poi = take_spectrum(pa, poi_name, folder_defect, results_poi)

    print("Run the saturation measurements")
    cobolt.set_laser_modulated_power(5)
    
    if not ple_is_here():
        res = pa.do_ple_scan(lines = 1)
    print("Fine optimization of the ple and confocal")
    res, results_poi = fine_optimize(pa, poi_name, folder_defect, results_poi)
    print("Run the saturation measurements")
    results_poi = run_saturation_measurement(pa, res, poi_name, folder_defect, results_poi)
    
    with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
        pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)

Find the defect  def1
Run spectrum
Run the saturation measurements
Fine optimization of the ple and confocal
Run the saturation measurements
NO, PLE 0.033474415845032635
NO, PLE 0.024530376789570707
NO, PLE 0.15451655179443635
NO, PLE 0.17266087438252142
NO, PLE 0.17266087438252142
NO, PLE 0.07732385042796786
NO, PLE 0.07732385042796786
NO, PLE 0.022436779884381863
NO, PLE 0.022436779884381863
NO, PLE 0.0361057355008616
NO, PLE 0.006786077576927618
NO, PLE 0.009910708908924426
NO, PLE 0.005514420080879434
NO, PLE 0.005514420080879434
NO, PLE 0.010876333519360126
NO, PLE 0.029553053146815
NO, PLE 0.021315836407955646
NO, PLE 0.021315836407955646
NO, PLE 0.008216189077942082
NO, PLE 0.007051713158866435
NO, PLE 0.005899489778865674
NO, PLE 0.005899489778865674
NO, PLE 0.008628390458879087
NO, PLE 0.01961682977947321
NO, PLE 0.005319239064132342
NO, PLE 0.005319239064132342
NO, PLE 0.01214459099955667
NO, PLE 0.012438860160652654
NO, PLE 0.007216024137078447
NO, PLE 0.007216024137078447
N

In [51]:
center_v

-7.4

In [53]:
folder = rf"C:\Users\yy3\Documents\data\Vlad\158\30-04\#1D\ROI2\{datetime.today().strftime('%d-%m-%Y')}"
os.makedirs(folder, exist_ok=True)
# folder = os.path.join(folder, r"auto")

center_v = laser_controller_remote.etalon_voltage
os.makedirs(folder, exist_ok=True)

poi_name = "def1"
results_poi = {}
results_poi[poi_name] = {}  


high_finesse_wavemeter_remote.start_acquisition()
# run throught the defects:
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(10)


os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)
#Go to the defect:

settings_confocal_refocus_fine()
switchlogic.set_state("ScanningMode", 'Wavemeter')
pa.set_resonant_power(power = 300)
pa.go_to_poi(poi_name, opt_times=2, ref_poi=None) 
#
# print("Find the defect ", poi_name)
# res = find_the_defect(pa, poi_name, folder_defect)




KeyboardInterrupt: 

In [56]:
print("Run spectrum")
results_poi = take_spectrum(pa, poi_name, folder_defect, results_poi)

Run spectrum


In [ ]:

print("Run the saturation measurements")
cobolt.set_laser_modulated_power(5)

if not ple_is_here():
    res = pa.do_ple_scan(lines = 1)
    if not ple_is_here():
        print("Adjusting the eta")
        res, results_poi = adjust_eta(pa, poi_name, folder_defect, results_poi, center_v)
if not ple_is_here():
    # return the center eta
    laser_controller_remote.etalon_voltage = center_v
    continue
print("Fine optimization of the ple and confocal")
res, results_poi = fine_optimize(pa, poi_name, folder_defect, results_poi)
print("Run the saturation measurements")
results_poi = run_saturation_measurement(pa, res, poi_name, folder_defect, results_poi)

with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
    pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [65]:
switchlogic.set_state("ScanningMode", 'NI')

In [67]:
res = pa.do_ple_scan(lines = 1)

In [69]:
# laser_controller_remote.etalon_voltage = center_v - 0.2

In [84]:
fine_sigma = 300
power_steps = [300 , 250, 200, 150, 130, 120,100, 90]
repumped_power = 200
no_repump_power = 200

os.makedirs(saturation_folder := os.path.join(folder_defect, "saturation"), exist_ok=True)

pa.set_resonant_power(power = power_steps[0])
res = pa.do_ple_scan(lines = 1)
pa.save_ple(tag = f"full_range_power_{power_steps[0]}", 
            poi_name=poi_name, folder_name=saturation_folder)



for idx, power in enumerate(power_steps):
    os.makedirs(power_folder := os.path.join(saturation_folder, f"{power}"), exist_ok=True)

    if power > repumped_power:
        cobolt.enable_modulated()
        cobolt.set_laser_modulated_power(5)
        pa.set_resonant_power(power = power)
        time.sleep(1)

        #finding ple in a small range
        res = pa.do_ple_scan(lines = 1, in_range=(
            res["center"].value - res["sigma"] * 10, 
            res["center"].value + res["sigma"] * 10    
        ))

        scan_sigma = FINE_SIGMA if res["sigma"] < fine_sigma / 3 else res["sigma"] * 3
        print("Scan sigma", scan_sigma)
        res = pa.do_ple_scan(lines = 1, in_range=(
            res["center"].value - scan_sigma, 
            res["center"].value + scan_sigma    
        ))

        pa.save_ple(tag = f"{power}",
                poi_name=poi_name, folder_name=power_folder)

        results_poi.update({"saturation": 
                            {f"{power}_repump":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr,
                                "center": res["center"].value
                                }
                                }})
        
    scan_sigma = fine_sigma if res["sigma"] < fine_sigma / 3 else res["sigma"] * 3
    print("Scan sigma no repump", scan_sigma)

    if (power < no_repump_power):
        os.makedirs(power_folder_norepump := os.path.join(power_folder, f"no_repump"), exist_ok=True)
        # check with initioalization
        cobolt.disable_modulated()
        pa.one_pulse_repump("violet")
        for i in range(3):
            pa.one_pulse_repump("violet")
            res_ = pa.do_ple_scan(lines = 1, in_range=scan_sigma)
            if not ple_is_here(rsquared_bound=0.4):
                continue
        if not ple_is_here(rsquared_bound=0.4):
                continue
        res_ = pa.do_ple_scan(lines = 5, in_range=scan_sigma)
        print("Saving no repump")
        pa.save_ple(tag = f"{power}_norepump",
            poi_name=poi_name, folder_name=power_folder_norepump)
        results_poi.update({"saturation": 
                        {f"{power}_norepump":
                            {"scan_data": ple_data_logic.last_saved_files_paths,
                            "sigma": res_["sigma"].value,
                            "sigma_stderr": res_["sigma"].stderr,
                            "center": res_["center"].value
                            }
                            }})

Scan sigma 2891.6101605174285
Scan sigma no repump 2918.530160776967
Scan sigma 1206.7786170433833
Scan sigma no repump 1542.2814734240756
Scan sigma 661.9666567489069
Scan sigma no repump 300
Scan sigma 300
Scan sigma no repump 300


KeyboardInterrupt: 

In [85]:
res = pa.do_ple_scan(lines = 1, in_range=(
    res["center"].value - scan_sigma, 
    res["center"].value + scan_sigma    
))

In [87]:
cobolt.disable_modulated()

In [ ]:
cobolt.enable_modulated()

In [91]:
cobolt.set_laser_modulated_power(1)

In [88]:
pa.one_pulse_repump("green")

In [92]:
settings_confocal_refocus_fine()

In [110]:
repumped_power = 120
no_repump_power = 150

In [189]:
#Proper
fine_sigma = 150
coarse_sigma = 4000
power_steps = [300 , 250, 200, 150, 130, 120,100, 90]
repumped_power = 200
no_repump_power = 180

os.makedirs(saturation_folder := os.path.join(folder_defect, "saturation"), exist_ok=True)

ple_gui._mw.frequencyDoubleSpinBox.setValue(500)
ple_gui._mw.resolutionDoubleSpinBox.setValue(2000)

cobolt.set_laser_modulated_power(2)
cobolt.enable_modulated()
for idx, power in enumerate(power_steps):
    pa.set_resonant_power(power = power)
    res = pa.do_ple_scan(lines = 1)
    pa.save_ple(tag = f"full_range_power_{power}", 
                poi_name=poi_name, folder_name=saturation_folder)
    for i in range(3):
        if not ple_is_here(rsquared_bound=0.5):
            continue
        res = pa.do_ple_scan(lines = 1)
    if not ple_is_here(rsquared_bound=0.5):
        break

# now we know the resonable power range for the resonant laser
#now we can do the saturation measurements without the cw repump

cobolt.disable_modulated()
for idx, power in enumerate(power_steps[idx:]):
    os.makedirs(power_folder := os.path.join(saturation_folder, f"{power}"), exist_ok=True)

    ple_gui._mw.frequencyDoubleSpinBox.setValue(250)

    # check if we see ple with the pre scan charge initialization
    pa.go_to_ple_target(laser_scanner_logic.scan_ranges["a"][0])
    pa.one_pulse_repump("violet")
    #finding ple in a large range
    for i in range(5):
        pa.one_pulse_repump("violet")
        res_ = pa.do_ple_scan(lines = 1, 
                                in_range=(
        res["center"].value - coarse_sigma, 
        res["center"].value + coarse_sigma    
                        ))
        if ple_is_here(rsquared_bound=0.5):
            break
    if not ple_is_here(rsquared_bound=0.5):
        print("NO PLE")
        # continue
    # Since ple is visible we can reassign the fit params
    res = res_ if ple_is_here() else res

    # Now we can try to zoom in
    for i in range(5):
        
        pa.one_pulse_repump("violet")
        res_ = pa.do_ple_scan(lines = 1, 
                                in_range=(
        res["center"].value - coarse_sigma/2, 
        res["center"].value + coarse_sigma/2    
                        ))
        if ple_is_here(rsquared_bound=0.5):
            break
    if not ple_is_here(rsquared_bound=0.5):
        print("NO PLE")
        # continue

    # Since ple is visible we can reassign the fit params
    res = res_ if ple_is_here() else res

    # now we can try no 
    for i in range(5):
            
        pa.one_pulse_repump("violet")
        res_ = pa.do_ple_scan(lines = 1, 
                                in_range=(
        res["center"].value - coarse_sigma/5, 
        res["center"].value + coarse_sigma/5    
                        ))
        if ple_is_here(rsquared_bound=0.5):
            break
    if not ple_is_here(rsquared_bound=0.5):
        print("NO PLE")
    else:
        res = res_ if ple_is_here() else res
    pa.one_pulse_repump("violet")

    # no repump stability
    res_ = pa.do_ple_scan(lines = 5, 
                                in_range=(
        res["center"].value - coarse_sigma/6, 
        res["center"].value + coarse_sigma/6    
                        ))

    print("Saving no repump")
    pa.save_ple(tag = f"{power}_norepump",
        poi_name=poi_name, folder_name=power_folder_norepump)
    results_poi.update({"saturation": 
                    {f"{power}_norepump":
                        {"scan_data": ple_data_logic.last_saved_files_paths,
                        "sigma": res_["sigma"].value,
                        "sigma_stderr": res_["sigma"].stderr,
                        "center": res_["center"].value
                        }
                        }})


NO, PLE 0.019929061671988046
NO, PLE 0.019929061671988046
NO, PLE 0.019929061671988046
NO, PLE 0.036181157079733905
NO, PLE 0.014198868714205282
NO, PLE 0.46455866556285663
NO, PLE 0.011972064112413472
NO, PLE 0.4539149472239279
NO, PLE 0.4539149472239279
NO PLE
NO, PLE 0.4539149472239279
Saving no repump
NO, PLE 0.01009056713434775
Saving no repump
NO, PLE 0.4578206937003334
Saving no repump
NO, PLE 0.2128815170656081
NO, PLE 0.20181951489665195
Saving no repump
NO, PLE 0.006886393658628087
NO, PLE 0.008017646514885235
NO, PLE 0.478478412333909
NO, PLE 0.4492204392542547
NO, PLE 0.07883847958967294
NO, PLE 0.23840387286531928
Saving no repump


In [194]:
folder = r"C:\Users\yy3\Documents\data\Vlad\158\30-04\#1D"

# 0 Search for a defect

In [351]:
switchlogic.set_state("ScanningMode", 'Wavemeter')

In [289]:
laser_controller_remote.etalon_voltage = center_v

In [336]:
pa.one_pulse_repump("violet")

In [339]:
pa.set_resonant_power(
    power = 300
)

In [355]:
300*300/5 /60 /60

5.0

In [363]:
cobolt.disable_modulated()

In [361]:
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(1)

In [340]:
switchlogic.set_state("ScanningMode", 'NI')

# 1 Full power full range

In [341]:
poi_name = "def14"


os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)

ple_gui._save_folderpath = folder_defect
ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)

In [342]:

cobolt.set_laser_modulated_power(2)
cobolt.enable_modulated()
pa.set_resonant_power(300)

res = pa.do_ple_scan(lines = 1)



pa.save_ple(tag = f"full_range_power_{power}", 
            poi_name=poi_name, folder_name=folder_defect)

In [343]:
pa.go_to_ple_target(res["center"].value)
time.sleep(1)
w1 = high_finesse_wavemeter_remote.get_current_wavelength()
print(1e-3 * c / w1, "nm")

619.2330194756765 nm


In [344]:
w1 = high_finesse_wavemeter_remote.get_current_wavelength()
print(1e-3 * c / w1, "nm")

619.2363643542185 nm


# 2 Small power small range. Pre scan repump

# 3 Take spectrum

In [350]:
print("Run spectrum")
ibeam_smart.setPower(60000)

results_poi = take_spectrum(pa, poi_name, folder_defect, results_poi)

Run spectrum


In [346]:
cobolt.disable_modulated()

In [347]:
pa.one_pulse_repump("violet")

In [348]:
cobolt.set_laser_modulated_power(3)

In [ ]:
res = pa.do_ple_scan(lines = 1, in_range=(
        res["center"].value - 2 * scan_sigma,
        res["center"].value + 2 * scan_sigma
    ))

In [243]:
spectrometer._mw.action_save_background.triggered.emit()

True

Run spectrum


In [ ]:
scan_sigma = 500

pa.set_resonant_power(110)
cobolt.disable_modulated()


for i in range(5):
    pa.go_to_ple_target(laser_scanner_logic.scan_ranges["a"][0])

    pa.one_pulse_repump("violet")
    res = pa.do_ple_scan(lines = 1, in_range=(
        res["center"].value - 5 * scan_sigma,
        res["center"].value + 5 * scan_sigma
    ))
    if ple_is_here(rsquared_bound=0.5):
        break

ple_gui._mw.frequencyDoubleSpinBox.setValue(1000)
for i in range(5):
    pa.one_pulse_repump("violet")
    res = pa.do_ple_scan(lines = 1, in_range=(
        res["center"].value - 2 * scan_sigma,
        res["center"].value + 2 * scan_sigma
    ))
    if ple_is_here(rsquared_bound=0.5):
        break



In [246]:
cobolt.enable_modulated()
pa.set_resonant_power(300)

In [253]:
cobolt.enable_modulated()

In [255]:
cobolt.set_laser_modulated_power(1)

In [ ]:

pa.save_ple(tag = f"low_power_{power}", 
            poi_name=poi_name, folder_name=folder_defect)

In [256]:
pa.one_pulse_repump("violet")

In [236]:
pa.one_pulse_repump("green")

619.2278328810866 nm


In [ ]:
FINE_SIGMA = 150
COARSE_SIGMA = 4000
POWER_STEPS = [300, 250, 200, 150, 130, 120, 100, 90]
REPUMPED_POWER = 200
NO_REPUMP_POWER = 180

def do_initial_calibration():
    os.makedirs(saturation_folder := os.path.join(folder_defect, "saturation"), exist_ok=True)

    ple_gui._mw.frequencyDoubleSpinBox.setValue(500)

    cobolt.set_laser_modulated_power(2)
    cobolt.enable_modulated()

    for idx, power in enumerate(POWER_STEPS):
        pa.set_resonant_power(power=power)
        res = pa.do_ple_scan(lines=1)
        pa.save_ple(tag=f"full_range_power_{power}", poi_name=poi_name, folder_name=saturation_folder)

        for i in range(3):
            if not ple_is_here(rsquared_bound=0.5):
                continue
            res = pa.do_ple_scan(lines=1)

        if not ple_is_here(rsquared_bound=0.5):
            break

    cobolt.disable_modulated()

def do_saturation_measurements():
    for idx, power in enumerate(POWER_STEPS[idx:]):
        os.makedirs(power_folder := os.path.join(saturation_folder, f"{power}"), exist_ok=True)

        ple_gui._mw.frequencyDoubleSpinBox.setValue(500)
        ple_gui._mw.resolutionDoubleSpinBox.setValue(2000)
        pa.go_to_ple_target(laser_scanner_logic.scan_ranges["a"][0])

        pa.set_resonant_power(power=power)
        pa.one_pulse_repump("violet")
        wait_for_event(lambda: ple_is_here(rsquared_bound=0.5))
        res = pa.do_ple_scan(lines=1, in_range=(res["center"].value - coarse_sigma, res["center"].value + coarse_sigma))

        for i in range(3):
            if not ple_is_here(rsquared_bound=0.5):
                continue
            pa.one_pulse_repump("violet")
            wait_for_event(lambda: ple_is_here(rsquared_bound=0.5))
            res = pa.do_ple_scan(lines=1, in_range=(res["center"].value - coarse_sigma, res["center"].value + coarse_sigma))

        if not ple_is_here(rsquared_bound=0.5):
            continue

        res_ = pa.do_ple_scan(lines=1, in_range=(res["center"].value - coarse_sigma * 2, res["center"].value + coarse_sigma * 2))

        scan_sigma = FINE_SIGMA if res["sigma"] < FINE_SIGMA / 3 else res["sigma"] * 3
        pa.one_pulse_repump("violet")
        wait_for_event(lambda: ple_is_here(rsquared_bound=0.5))
        res_ = pa.do_ple_scan(lines=1, in_range=(res["center"].value - scan_sigma * 2, res["center"].value + scan_sigma * 2))
        res = res_ if ple_is_here() else res

        pa.one_pulse_repump("violet")
        wait_for_event(lambda: ple_is_here(rsquared_bound=0.5))
        res_ = pa.do_ple_scan(lines=1, in_range=(res["center"].value - scan_sigma, res["center"].value + scan_sigma))
        res = res_ if ple_is_here() else res

        pa.one_pulse_repump("violet")
        wait_for_event(lambda: ple_is_here(rsquared_bound=0.5))
        res_ = pa.do_ple_scan(lines=5, in_range=(res["center"].value - scan_sigma, res["center"].value + scan_sigma))

        print("Saving no repump")
        pa.save_ple(tag=f"{power}_norepump", poi_name=poi_name, folder_name=power_folder_norepump)

        results_poi.update({
            "saturation": {
                f"{power}_norepump": {
                    "scan_data": ple_data_logic.last_saved_files_paths,
                    "sigma": res_["sigma"].value,
                    "sigma_stderr": res_["sigma"].stderr,
                    "center": res_["center"].value
                }
            }
        })

def wait_for_event(event_func, timeout=10):
    start_time = time.time()
    while not event_func():
        if time.time() - start_time > timeout:
            raise TimeoutError("Timed out waiting for event")
        time.sleep(0.1)


In [157]:

os.makedirs(power_folder := os.path.join(saturation_folder, f"{power}"), exist_ok=True)

ple_gui._mw.frequencyDoubleSpinBox.setValue(500)
# cobolt.enable_modulated()
cobolt.disable_modulated()

pa.go_to_ple_target(laser_scanner_logic.scan_ranges["a"][0])

pa.set_resonant_power(power = power)
pa.one_pulse_repump("green")
time.sleep(0.5)
pa.one_pulse_repump("green")
#finding ple in a large range
res_ = pa.do_ple_scan(lines = 1, in_range=(
    res["center"].value - coarse_sigma, 
    res["center"].value + coarse_sigma    
))
pa.one_pulse_repump("green")
time.sleep(0.5)
if ple_is_here(rsquared_bound=0.3):
    res_ = pa.do_ple_scan(lines = 1, in_range=(
        res["center"].value - coarse_sigma * 2, 
        res["center"].value + coarse_sigma * 2    
))
else:
    cobolt.enable_modulated()
    res_ = pa.do_ple_scan(lines = 1, in_range=(
    res["center"].value - coarse_sigma * 2, 
    res["center"].value + coarse_sigma * 2    
    ))
if ple_is_here(rsquared_bound=0.5):
    res = res_

KeyboardInterrupt: 

In [156]:
ple_gui._mw.Pulsed_widget.prescan_checkBox.setChecked(True)

In [153]:
pa.one_pulse_repump("violet")

In [ ]:

if (power < no_repump_power):
    os.makedirs(power_folder_norepump := os.path.join(power_folder, f"no_repump"), exist_ok=True)
    # check with initioalization
    cobolt.disable_modulated()
    pa.one_pulse_repump("violet")
    # search for the center
    for i in range(3):
        pa.one_pulse_repump("violet")
        time.sleep(0.5)
        res_ = pa.do_ple_scan(lines = 1, 
                                in_range=(
                        res["center"].value - scan_sigma* 2, 
                        res["center"].value + scan_sigma * 2    
                                            )
                                            )
        if not ple_is_here(rsquared_bound=0.25):
            pass
    # no ple whatsoever , skip
    if not ple_is_here(rsquared_bound=0.25):
            pass
    res_ = pa.do_ple_scan(lines = 5, in_range=(
        res["center"].value - scan_sigma, 
        res["center"].value + scan_sigma    
    ))
    print("Saving no repump")
    pa.save_ple(tag = f"{power}_norepump",
        poi_name=poi_name, folder_name=power_folder_norepump)
    results_poi.update({"saturation": 
                    {f"{power}_norepump":
                        {"scan_data": ple_data_logic.last_saved_files_paths,
                        "sigma": res_["sigma"].value,
                        "sigma_stderr": res_["sigma"].stderr,
                        "center": res_["center"].value
                        }
                        }})

In [146]:
 ple_gui.fit_result[1].rsquared

0.6729438900240361

In [147]:
pa.one_pulse_repump("violet")

In [111]:

if power > repumped_power:
    cobolt.enable_modulated()
    #finding ple in a small range
    scan_sigma = 5 * fine_sigma if res["sigma"] < fine_sigma else res["sigma"] * 5
    print("Scan sigma", scan_sigma)
    res_f1 = pa.do_ple_scan(lines = 1, in_range=(
        res["center"].value - scan_sigma, 
        res["center"].value + scan_sigma    
    ))

    if ple_is_here(rsquared_bound=0.5):
        print("Ple is here")
        res = res_f1
    else:
        print("Ple is here not")
        res = pa.do_ple_scan(lines = 1, in_range=(
            res["center"].value - scan_sigma * 3, 
            res["center"].value + scan_sigma * 3    
    ))
    pa.save_ple(tag = f"{power}",
            poi_name=poi_name, folder_name=power_folder)

    results_poi.update({"saturation": 
                        {f"{power}_repump":
                            {"scan_data": ple_data_logic.last_saved_files_paths,
                            "sigma": res["sigma"].value,
                            "sigma_stderr": res["sigma"].stderr,
                            "center": res["center"].value
                            }
                            }})

Scan sigma 750
NO, PLE 0.28287273360114784
Ple is here not


In [112]:
scan_sigma = fine_sigma if res["sigma"] < fine_sigma else res["sigma"] * 2
print("Scan sigma no repump", scan_sigma)


Scan sigma no repump 150


In [116]:
no_repump_power = 201

NO, PLE 0.014742829106390576


KeyboardInterrupt: 

In [96]:
cobolt.enable_modulated()